In [ ]:
# Data processing
import iris
import iris.analysis
import iris.coord_categorisation
import warnings
warnings.filterwarnings('ignore', module='iris')
import numpy as np
from pathlib import Path
from scipy import stats
# Visualization
import cartopy.util
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.ticker import FuncFormatter
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)
    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
def m2km(x, pos):
    '''Convert meters to kilometers when plotting axis labels'''
    return int(x*1e-3)
def custom_ctcklbls(x, pos):
    '''Custom format for colorbar tick labels'''
    return '{:1.1f}'.format(x)
plt.rcParams['mathtext.default'] = 'regular'

In [ ]:
# Choose variable
dir_var = 'meono2'
str_var = '$MeONO_2$'
coeff_var = (28.97/77.0394)*1e12 # coeff to convert mass mixing ratio to volume mixing ratio where M dry air = 28.97 [g mol-1]
# Choose jobs
job = 'xojnh'
if job == 'xojng': exp = 'BASE'
elif job == 'xojnh': exp = 'CHEM'
elif job == 'xojni': exp = 'OCEAN'
elif job == 'xojnc': exp = 'BB';
elif job == 'xojnl': exp = 'ALL'
# Read data
path_to_ukca = Path('../data')
ijob = iris.load_cube(str(path_to_ukca / job / f'{job}_chem_15yrs.pp'), 'mass_fraction_of_methyl_nitrate_in_air')*coeff_var
# Path to figures
path_to_figs = Path('../../../results') / job / dir_var
path_to_figs.mkdir(exist_ok=True)

In [ ]:
savefig = True
publish = True
if publish:
    mpl.rcParams['xtick.labelsize'] = 25
    mpl.rcParams['ytick.labelsize'] = 25
    mpl.rcParams['axes.titlesize'] = 40
    mpl.rcParams['axes.labelsize'] = 30
    plt.rcParams['font.size'] = 30

In [ ]:
# Select 8 years
ijob = ijob[24:120,...]
# Add season and year coordinates
iris.coord_categorisation.add_season(ijob, 'time', name='season')
iris.coord_categorisation.add_season_year(ijob, 'time', name='year')
str_djf = 'DJF'; str_mam = 'MAM'; str_jja = 'JJA'; str_son = 'SON'

#### Boundary layer seasonal means

In [ ]:
# Extract time series of boundary layer (0-2 km) seasonal means
ijob_pbl_djf = (ijob.extract(iris.Constraint(season='djf')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,0:10,...]).collapsed('level_height', iris.analysis.MEAN)
ijob_pbl_mam = (ijob.extract(iris.Constraint(season='mam')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,0:10,...]).collapsed('level_height', iris.analysis.MEAN)
ijob_pbl_jja = (ijob.extract(iris.Constraint(season='jja')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,0:10,...]).collapsed('level_height', iris.analysis.MEAN)
ijob_pbl_son = (ijob.extract(iris.Constraint(season='son')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,0:10,...]).collapsed('level_height', iris.analysis.MEAN)
# Calculate boundary layer seasonal means
ijob_pbl_djf_mean = ijob_pbl_djf.collapsed('year', iris.analysis.MEAN)
ijob_pbl_mam_mean = ijob_pbl_mam.collapsed('year', iris.analysis.MEAN)
ijob_pbl_jja_mean = ijob_pbl_jja.collapsed('year', iris.analysis.MEAN)
ijob_pbl_son_mean = ijob_pbl_son.collapsed('year', iris.analysis.MEAN)
# Add cyclic point for plotting on a global map
cyc_ijob_pbl_djf_mean, cyclic_lons = cartopy.util.add_cyclic_point(ijob_pbl_djf_mean.data, coord=ijob_pbl_djf_mean.coord('longitude').points)
cyc_ijob_pbl_mam_mean = cartopy.util.add_cyclic_point(ijob_pbl_mam_mean.data)
cyc_ijob_pbl_jja_mean = cartopy.util.add_cyclic_point(ijob_pbl_jja_mean.data)
cyc_ijob_pbl_son_mean = cartopy.util.add_cyclic_point(ijob_pbl_son_mean.data)
# Find max boundary layer seasonal mean
print(np.max(ijob_pbl_djf_mean.data))
print(np.max(ijob_pbl_mam_mean.data))
print(np.max(ijob_pbl_jja_mean.data))
print(np.max(ijob_pbl_son_mean.data))

h: 18.90242
17.880545
21.308216
18.330566

In [ ]:
# Boundary layer seasonal mean difference plotting parameters
pbl_mean_cf_kwargs = dict(transform=ccrs.PlateCarree(), levels=np.arange(0,60,10))

In [ ]:
lats = ijob.coord('latitude').points

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(18,18), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0)), facecolor='w')
p00 = ax[0,0].contourf(cyclic_lons, lats, cyc_ijob_pbl_djf_mean.data, **pbl_mean_cf_kwargs)
ax[0,1].contourf(cyclic_lons, lats, cyc_ijob_pbl_mam_mean.data, **pbl_mean_cf_kwargs)
ax[1,0].contourf(cyclic_lons, lats, cyc_ijob_pbl_jja_mean.data, **pbl_mean_cf_kwargs)
ax[1,1].contourf(cyclic_lons, lats, cyc_ijob_pbl_son_mean.data, **pbl_mean_cf_kwargs)
fig.subplots_adjust(hspace=-0.9, wspace=-0.5)
fig.tight_layout()
cax0 = fig.add_axes([0.35, 0.21, 0.3, 0.01])
fig.colorbar(p00, cax=cax0, orientation='horizontal', label=f'{str_var}, pptv')
fig.suptitle(f'Boundary layer (0-2 km)\n{exp}', y=0.8, weight='bold')
ax[0,0].set_title(str_djf)
ax[0,1].set_title(str_mam)
ax[1,0].set_title(str_jja)
ax[1,1].set_title(str_son)
for iax in ax.flatten(): 
    iax.set_global()
    iax.coastlines()
if savefig: fig.savefig(path_to_figs / f'{job}_{dir_var}_pbl_seas_mean_meono2.png', bbox_inches='tight')

In [ ]:
break

#### Free troposphere seasonal means

In [ ]:
# Extract time series of free troposphere (5-10 km) seasonal means
ijob_fre_djf = (ijob.extract(iris.Constraint(season='djf')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,15:23,...]).collapsed('level_height', iris.analysis.MEAN)
ijob_fre_mam = (ijob.extract(iris.Constraint(season='mam')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,15:23,...]).collapsed('level_height', iris.analysis.MEAN)
ijob_fre_jja = (ijob.extract(iris.Constraint(season='jja')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,15:23,...]).collapsed('level_height', iris.analysis.MEAN)
ijob_fre_son = (ijob.extract(iris.Constraint(season='son')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,15:23,...]).collapsed('level_height', iris.analysis.MEAN)
# Calculate free troposphere seasonal mean differences
ijob_fre_djf_mean = ijob_fre_djf.collapsed('year', iris.analysis.MEAN)
ijob_fre_mam_mean = ijob_fre_mam.collapsed('year', iris.analysis.MEAN)
ijob_fre_jja_mean = ijob_fre_jja.collapsed('year', iris.analysis.MEAN)
ijob_fre_son_mean = ijob_fre_son.collapsed('year', iris.analysis.MEAN)
# Add cyclic point for plotting on a global map
cyc_ijob_fre_djf_mean, cyclic_lons = cartopy.util.add_cyclic_point(ijob_fre_djf_mean.data, coord=ijob_fre_djf_mean.coord('longitude').points)
cyc_ijob_fre_mam_mean = cartopy.util.add_cyclic_point(ijob_fre_mam_mean.data)
cyc_ijob_fre_jja_mean = cartopy.util.add_cyclic_point(ijob_fre_jja_mean.data)
cyc_ijob_fre_son_mean = cartopy.util.add_cyclic_point(ijob_fre_son_mean.data)
# Find max free troposphere seasonal mean
print(np.max(ijob_fre_djf_mean.data))
print(np.max(ijob_fre_mam_mean.data))
print(np.max(ijob_fre_jja_mean.data))
print(np.max(ijob_fre_son_mean.data))

In [ ]:
# Free troposphere seasonal mean difference plotting parameters
fre_mean_cf_kwargs = dict(transform=ccrs.PlateCarree(), levels=np.arange(0,175,25))
fre_diff_cf_kwargs = dict(transform=ccrs.PlateCarree(), cmap='RdBu_r')
fre_pval_sc_kwargs = dict(transform=ccrs.PlateCarree(), s=10, c='k', marker='.', alpha=0.2)

In [ ]:
# Find min and max absolute free troposphere seasonal mean difference
print(min([np.min(cyc_fre_djf_diff.data), np.min(cyc_fre_mam_diff.data), np.min(cyc_fre_jja_diff.data), np.min(cyc_fre_son_diff.data)]))
print(max([np.max(cyc_fre_djf_diff.data), np.max(cyc_fre_mam_diff.data), np.max(cyc_fre_jja_diff.data), np.max(cyc_fre_son_diff.data)]))
# Find min and max % free troposphere seasonal mean difference
print(min([np.min(cyc_fre_djf_diff.data/cyc_base_fre_djf_mean.data*100), np.min(cyc_fre_mam_diff.data/cyc_base_fre_mam_mean.data*100), 
           np.min(cyc_fre_jja_diff.data/cyc_base_fre_jja_mean.data*100), np.min(cyc_fre_son_diff.data/cyc_base_fre_son_mean.data*100)]))
print(max([np.max(cyc_fre_djf_diff.data/cyc_base_fre_djf_mean.data*100), np.max(cyc_fre_mam_diff.data/cyc_base_fre_mam_mean.data*100), 
           np.max(cyc_fre_jja_diff.data/cyc_base_fre_jja_mean.data*100), np.max(cyc_fre_son_diff.data/cyc_base_fre_son_mean.data*100)]))

In [ ]:
# Additional plotting parameteres for free troposphere seasonal mean summary
fre_glb_absdiff_cf_kwargs = dict(transform=ccrs.PlateCarree(), cmap='RdBu_r', levels=np.arange(-12,14,2))
fre_glb_pctdiff_cf_kwargs = dict(transform=ccrs.PlateCarree(), cmap='RdBu_r', levels=np.arange(-14,16,2))#, extend='both')

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=3, figsize=(12,12), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0)), facecolor='w')
ax[0,0].contourf(cyclic_lons, lats, cyc_base_fre_djf_mean.data, **fre_mean_cf_kwargs)
ax[0,1].contourf(cyclic_lons, lats, cyc_fre_djf_diff.data, **fre_glb_absdiff_cf_kwargs)
ax[0,2].contourf(cyclic_lons, lats, cyc_fre_djf_diff.data/cyc_base_fre_djf_mean.data*100, **fre_glb_pctdiff_cf_kwargs)

ax[1,0].contourf(cyclic_lons, lats, cyc_base_fre_mam_mean.data, **fre_mean_cf_kwargs)
ax[1,1].contourf(cyclic_lons, lats, cyc_fre_mam_diff.data, **fre_glb_absdiff_cf_kwargs)
ax[1,2].contourf(cyclic_lons, lats, cyc_fre_mam_diff.data/cyc_base_fre_mam_mean.data*100, **fre_glb_pctdiff_cf_kwargs)

ax[2,0].contourf(cyclic_lons, lats, cyc_base_fre_jja_mean.data, **fre_mean_cf_kwargs)
ax[2,1].contourf(cyclic_lons, lats, cyc_fre_jja_diff.data, **fre_glb_absdiff_cf_kwargs)
ax[2,2].contourf(cyclic_lons, lats, cyc_fre_jja_diff.data/cyc_base_fre_jja_mean.data*100, **fre_glb_pctdiff_cf_kwargs)

p30 = ax[3,0].contourf(cyclic_lons, lats, cyc_base_fre_son_mean.data, **fre_mean_cf_kwargs)
p31 = ax[3,1].contourf(cyclic_lons, lats, cyc_fre_son_diff.data, **fre_glb_absdiff_cf_kwargs)
p32 = ax[3,2].contourf(cyclic_lons, lats, cyc_fre_son_diff.data/cyc_base_fre_son_mean.data*100, **fre_glb_pctdiff_cf_kwargs)

fig.subplots_adjust(hspace=-0.8, wspace=-0.5)
fig.tight_layout()
cax0 = fig.add_axes([0.02, 0.12, 0.3, 0.01])
cax1 = fig.add_axes([0.35, 0.12, 0.3, 0.01])
cax2 = fig.add_axes([0.68, 0.12, 0.3, 0.01])
fig.colorbar(p30, cax=cax0, orientation='horizontal', label=f'{str_var}, ppbv')
fig.colorbar(p31, cax=cax1, orientation='horizontal', label=f'$\Delta${str_var}, ppbv')
fig.colorbar(p32, cax=cax2, orientation='horizontal', label=f'$\Delta${str_var}, %')
fig.text(0.02, 0.84, 'DJF')#, fontsize=12)
fig.text(0.02, 0.66, 'MAM')#, fontsize=12)
fig.text(0.02, 0.48, 'JJA')#, fontsize=12)
fig.text(0.02, 0.31, 'SON')#, fontsize=12)
fig.suptitle('Free troposphere (5-10 km)', y=0.9, weight='bold')
ax[0,0].set_title(f'{base_plt}')
ax[0,1].set_title(f'{sens_plt}-{base_plt}')
ax[0,2].set_title(f'{sens_plt}-{base_plt}')
for iax in ax[0,1:3].flatten(): iax.scatter(*stipple_fre(fre_djf_diff, fre_djf_p, fdr_fre), **fre_pval_sc_kwargs)
for iax in ax[1,1:3].flatten(): iax.scatter(*stipple_fre(fre_mam_diff, fre_mam_p, fdr_fre), **fre_pval_sc_kwargs)
for iax in ax[2,1:3].flatten(): iax.scatter(*stipple_fre(fre_jja_diff, fre_jja_p, fdr_fre), **fre_pval_sc_kwargs)
for iax in ax[3,1:3].flatten(): iax.scatter(*stipple_fre(fre_son_diff, fre_son_p, fdr_fre), **fre_pval_sc_kwargs)
for iax in ax.flatten(): iax.set_global()
for iax in ax[:,0].flatten(): iax.coastlines(color='k')
for iax in ax[0:4,1:].flatten(): iax.coastlines(color='grey')
if savefig: fig.savefig(path_to_figs / f'{dir_var}_fre_seas_mean_all.png', bbox_inches='tight')